# RAG demo with OpenVINO Model Server and langchain
This demo shows how to use Retrieval Augmented Generation with langchain and gen AI endpoint from OpenVINO Model Server.

It employs the `chat/completion` and `embeddings` and `rerank` endpoints.

It assumes the model server is already deployed on the same machine on port 8000 with:

OpenVINO models:
 `OpenVINO/Qwen3-8B-int4-ov` for `chat/completions` and `OpenVINO/bge-base-en-v1.5-fp16-ov` for `embeddings` and `OpenVINO/bge-reranker-base-fp16-ov` for `rerank` endpoint.

or
Converted models:
 `meta-llama/Meta-Llama-3-8B-Instruct` for `chat/completions` and `Alibaba-NLP/gte-large-en-v1.5` for `embeddings` and `BAAI/bge-reranker-large` for `rerank` endpoint. 

Check https://github.com/openvinotoolkit/model_server/tree/releases/2025/2/demos/continuous_batching/rag/README.md to see how they can be deployed.
LLM model, embeddings and rerank can be on hosted on the same model server instance or separately as needed.
openai_api_base , base_url parameters with the target url and model names in the commands might need to be adjusted. 



In [1]:
%pip install -q --upgrade pip
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ipywidgets import RadioButtons, Text, VBox
options = ["OpenVINO models", "Converted models"]
output_text = Text()
def on_radio_change(change):
    output_text.value = change['new']

selection = RadioButtons(options=options, value="OpenVINO models", description="OVMS running with:", disabled=False)
selection.observe(on_radio_change, names='value')
VBox([selection, output_text])

In [4]:
print(output_text)
if output_text.value == "OpenVINO models":
    embeddings_model = "OpenVINO/bge-base-en-v1.5-fp16-ov"
    rerank_model = "OpenVINO/bge-reranker-base-fp16-ov"
    chat_model = "OpenVINO/Qwen3-8B-int4-ov"
else:
    embeddings_model = "Alibaba-NLP/gte-large-en-v1.5"
    rerank_model = "BAAI/bge-reranker-large"
    chat_model = "meta-llama/Meta-Llama-3-8B-Instruct"
    

Text(value='OpenVINO models')


In [5]:
import os

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

# Document Splitter
from typing import List
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader, )

from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

The documents to scan with knowledge context are to be placed in ./docs folder

In [6]:
TARGET_FOLDER = "./docs/"

TEXT_SPLITERS = {
    "Character": CharacterTextSplitter,
    "RecursiveCharacter": RecursiveCharacterTextSplitter,
    "Markdown": MarkdownTextSplitter,
}

LOADERS = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

In [7]:
!curl https://docs.openvino.ai/2025/model-server/ovms_what_is_openvino_model_server.html --create-dirs -o ./docs/ovms_what_is_openvino_model_server.html
!curl https://docs.openvino.ai/2025/model-server/ovms_docs_metrics.html -o ./docs/ovms_docs_metrics.html
!curl https://docs.openvino.ai/2025/model-server/ovms_docs_streaming_endpoints.html -o ./docs/ovms_docs_streaming_endpoints.html
!curl https://docs.openvino.ai/2025/model-server/ovms_docs_target_devices.html -o ./docs/ovms_docs_target_devices.html


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  742k    0  742k    0     0  1253k      0 --:--:-- --:--:-- --:--:-- 1251k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  768k    0  768k    0     0  1433k      0 --:--:-- --:--:-- --:--:-- 1435k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:

In [8]:
def load_single_document(file_path: str) -> List[Document]:
    """
    helper for loading a single document

    Params:
      file_path: document path
    Returns:
      documents loaded

    """
    ext = "." + file_path.rsplit(".", 1)[-1]
    if ext in LOADERS:
        loader_class, loader_args = LOADERS[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"File does not exist '{ext}'")

In [9]:
embeddings = OpenAIEmbeddings(
    model=embeddings_model,
    api_key="unused",
    tiktoken_enabled=False,
    base_url="http://localhost:8000/v3",
    embedding_ctx_length=8190,  # 8190 is the model max context length subtracted by 2 special tokens 
)


In [10]:
documents = []
for file_path in os.listdir(TARGET_FOLDER):
    if not file_path.endswith('.html'):
        continue
    abs_path = os.path.join(TARGET_FOLDER, file_path)
    print(f"Reading document {abs_path}...", flush=True)
    documents.extend(load_single_document(abs_path))

Reading document ./docs/ovms_docs_metrics.html...
Reading document ./docs/ovms_docs_streaming_endpoints.html...
Reading document ./docs/ovms_docs_target_devices.html...
Reading document ./docs/ovms_what_is_openvino_model_server.html...


In [11]:
spliter_name = "RecursiveCharacter"  # PARAM
chunk_size=1000  # PARAM
chunk_overlap=200  # PARAM
text_splitter = TEXT_SPLITERS[spliter_name](chunk_size=chunk_size, chunk_overlap=chunk_overlap)

texts = text_splitter.split_documents(documents)



In [13]:
try:
    db.delete_collection()
except:
    pass
db = FAISS.from_documents(texts, embeddings)  # This command populates vector store with embeddings

In [14]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

The commands below can be used to test the retriever. It can report the content for a given query.

In [15]:
vector_search_top_k = 5
retriever = db.as_retriever(search_kwargs={"k": vector_search_top_k})

retrieved_docs = retriever.invoke("Which metrics are supported in the model server? Give examples.")
pretty_print_docs(retrieved_docs)

Document 1:

Metrics#

Introduction#

This document describes how to use metrics endpoint in the OpenVINO Model Server. They can be applied for:

Providing performance and utilization statistics for monitoring and benchmarking purposes

Auto scaling of the model server instances in Kubernetes and OpenShift based on application related metrics

Built-in metrics allow tracking the performance without any extra logic on the client side or using network traffic monitoring tools like load balancers or reverse-proxies.

It also exposes metrics which are not related to the network traffic.

For example, statistics of the inference execution queue, model runtime parameters etc. They can also track the usage based on model version, API type or requested endpoint methods.

OpenVINO Model Server metrics are compatible with Prometheus standard

They are exposed on the /metrics endpoint.

Available metrics families#

Metrics from default list are enabled with the metrics_enable flag or json configu


Below the document compressor is used to filter the documents to the most relevant for the given query. It employs rerank endpoint in the model server and cohere client.
In the response is reported a list of documents limited to top_n.

In [16]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
import cohere
co = cohere.ClientV2(
    api_key="no_used",
    base_url="http://localhost:8000/v3/",
)
compressor = CohereRerank(model=rerank_model, client=co, top_n=1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "Which metrics are supported in the model server? Give examples.",
)
pretty_print_docs(compressed_docs)

Document 1:

Metrics#

Introduction#

This document describes how to use metrics endpoint in the OpenVINO Model Server. They can be applied for:

Providing performance and utilization statistics for monitoring and benchmarking purposes

Auto scaling of the model server instances in Kubernetes and OpenShift based on application related metrics

Built-in metrics allow tracking the performance without any extra logic on the client side or using network traffic monitoring tools like load balancers or reverse-proxies.

It also exposes metrics which are not related to the network traffic.

For example, statistics of the inference execution queue, model runtime parameters etc. They can also track the usage based on model version, API type or requested endpoint methods.

OpenVINO Model Server metrics are compatible with Prometheus standard

They are exposed on the /metrics endpoint.

Available metrics families#

Metrics from default list are enabled with the metrics_enable flag or json configu

Finally, LLM component needs to be configured. Here will be used chat/completions endpoint from the model server.
Change the base url and model name depending on the model server deployment and configuration. It is important to use /v3/ part which is specific for the OpenVINO Model Server

In [17]:
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v3",
    model_name=chat_model,
    verbose=True
)


With all the building blocks defined, the RAG chain is established to link all the components. 

In [18]:

prompt=PromptTemplate(input_variables=['context', 'question'], 
                      template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:")

print("prompt", prompt)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

prompt input_variables=['context', 'question'] input_types={} partial_variables={} template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:"


Below you can start the RAG chain using your own query. It will call the embedding model first, retrieve the relevant context and pass it to the LLM endpoint in a single request

In [19]:
for chunk in rag_chain.stream("Which metrics are supported in the model server? Give examples."):
    print(chunk, end="", flush=True)

<think>
Okay, the user is asking which metrics are supported in the model server and wants examples. Let me look at the context provided.

The context mentions that the OpenVINO Model Server has a metrics endpoint. It says the metrics can be used for monitoring and benchmarking, auto scaling in Kubernetes/OpenShift, and tracking performance without extra client logic. They also mention that some metrics aren't related to network traffic, like inference execution queue stats and model runtime parameters. Additionally, metrics can track usage based on model version, API type, or endpoint methods. The metrics are compatible with Prometheus and exposed on /metrics. 

The available metrics families are enabled via metrics_enable flag or JSON config. But the context doesn't list the specific metrics. The example given includes inference execution queue stats and model runtime parameters. The user wants examples, so I should mention those. Also, since the metrics are Prometheus-compatible, ma